In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# define chassis pickup points

# F/R - front/rear of car
# r/l - right/left
# U/L - upper/lower
# 1/2/3 - 1=front/2=rear/3=upright of the specific wishbone

# Should make control arm and upright classes to make this easier

# front right corner
FrU1 = np.array([1,0,0])
FrU2 = np.array([0,0,0])
FrU3 = np.array([.5,-.5,0])

# front right upright
joint_dist = .5
# define coords for ball joints and toe link so u can apply the same axis/ rotation checks to find 
# how far the upright rotated
# also define axle somehow to do same kind of math for wheel angles

In [ ]:
# axis of rotation
def axis_of_rot(p1: np.array,p2: np.array):
    vec = p1 - p2
    mag = np.linalg.norm(vec)

    if mag == 0:
        raise ValueError("axis_of_rot unit vec length = 0")
    
    unit_vec = vec / mag
    return unit_vec

# rotate a point about an axis
def rotation_about_axis(point, axis_point1, axis_point2, theta):
    # Step 1: Get the unit vector of the axis
    axis = axis_of_rot(axis_point1, axis_point2)

    # Step 2: Translate the point so axis_point1 becomes the origin
    translated_point = point - axis_point1

    # Step 3: Build the rotation matrix using Rodrigues' rotation formula
    ux, uy, uz = axis
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    one_minus_cos = 1 - cos_t

    R = np.array([
        [cos_t + ux**2 * one_minus_cos,
         ux * uy * one_minus_cos - uz * sin_t,
         ux * uz * one_minus_cos + uy * sin_t],

        [uy * ux * one_minus_cos + uz * sin_t,
         cos_t + uy**2 * one_minus_cos,
         uy * uz * one_minus_cos - ux * sin_t],

        [uz * ux * one_minus_cos - uy * sin_t,
         uz * uy * one_minus_cos + ux * sin_t,
         cos_t + uz**2 * one_minus_cos]
    ])

    # Step 4: Rotate the translated point
    rotated_translated_point = R @ translated_point

    # Step 5: Translate back to the original coordinate system
    rotated_point = rotated_translated_point + axis_point1

    return rotated_point